In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [2]:
reportLoc = r"C:\\Users\\kevin\\Documents\\AUTOM8\\Nutribins\\NutribinsLLC-Joliet-Bimonthly.xlsx"
activityLoc = r"C:\\Users\\kevin\\Documents\\AUTOM8\\Nutribins\\AR\\report aug.xlsx"

In [3]:
ExcelWorkbook = load_workbook(r"C:\\Users\\kevin\\Documents\\AUTOM8\\Nutribins\\random.xlsx")
 
# Generating the writer engine
writer = pd.ExcelWriter(r"C:\\Users\\kevin\\Documents\\AUTOM8\\Nutribins\\random.xlsx", engine = 'openpyxl')
 
# Assigning the workbook to the writer engine
writer.book = ExcelWorkbook
 
 
# Creating first dataframe
DataSample1= [[10,'value1'], 
             [20,'value2'], 
             [30,'value3']]
SimpleDataFrame1=pd.DataFrame(data=DataSample1, columns=['Col1','Col2'])
print(SimpleDataFrame1)
 
# Creating second dataframe
DataSample2= [[100,'A'], 
             [200,'B'], 
             [300,'C']]
SimpleDataFrame2=pd.DataFrame(data=DataSample2, columns=['colA','colB'])
print(SimpleDataFrame2)
 
# Adding the DataFrames to the excel as a new sheet
SimpleDataFrame1.to_excel(writer, sheet_name = 'Data1')
SimpleDataFrame2.to_excel(writer, sheet_name = 'Data2')
writer.save()
writer.close()

   Col1    Col2
0    10  value1
1    20  value2
2    30  value3
   colA colB
0   100    A
1   200    B
2   300    C


In [4]:
report = pd.read_excel(reportLoc)
rows = report['Description'].tolist()
for count, name in enumerate(rows):
    rows[count] = name.lower()

print(rows)

def nameMatch(name):
    for count in range(len(rows)):
        if rows[count] == name.lower():
            return count

def dataCopy(dataframe, sheetName):
    book = load_workbook(reportLoc)
    writer = pd.ExcelWriter(reportLoc, engine = 'openpyxl')
    writer.book = book

    if sheetName in writer.sheets:
        dataframe.to_excel(writer, startrow = writer.sheets[sheetName].max_row, index = False, header = False)
    else:
        dataframe.to_excel(writer, sheet_name = sheetName, index = False)

    writer.save()
    writer.close()

['handling pick per pallet', 'handling pick per case, weightrangepercase,0 - 30;', 'handling order processing per order, ordertype,rg;outbound shipmethod,ltl;', 'handling pick per piece, weightrangepereach,over 30;', 'materials & other charges-pallet charge grade b', 'materials & other charges-stretch wrap', 'storage income initial storage per pallet, locationtype,1 high;palletsize,none;', 'hourly labor $ / hour', 'slip sheet', 'customized shipping documents', 'recurring storage']


In [5]:
# Handling Pick per Pallet
df = pd.read_excel(io=activityLoc, sheet_name='Pick Task')
sum = df['PALLETPICKQTY'].sum()

index = nameMatch('Handling Pick per Pallet')

report['WISE Qty'][index] = sum
dataCopy(df, 'PALLET PICK')

c:\Users\kevin\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


KeyboardInterrupt: 

In [ ]:
# Handling Pick per Case, WeightRangePerCase,0 - 30;
df = pd.read_excel(io=activityLoc, sheet_name='Pick Task')
df2 = df[(df['INDIVIDUAL ITEM WGT'] >= 0) & (df['INDIVIDUAL ITEM WGT'] <= 30)]
sum = df2['CASEPICKQTY'].sum()

index = nameMatch('Handling Pick per Case, WeightRangePerCase,0 - 30;')

report['WISE Qty'][index] = sum
dataCopy(df2, reportLoc, 'HANDLING PICK PER CASE 0-30')

In [ ]:
# handling order processing per order, ordertype,rg;outbound shipmethod,ltl;
df = pd.read_excel(io=activityLoc, sheet_name='Order & Receipt')
df2 = df[df['Ship Method'].isin(['LTL'])]
df2 = df2[df2['RN/DN'].str.contains('DN', case=False, na=False)]
count = df2['RN/DN'].count()

index = nameMatch('handling order processing per order, ordertype,rg;outbound shipmethod,ltl;')
report['WISE Qty'][index] = count

In [ ]:
# handling pick per piece, weightrangepereach,over 30;
df = pd.read_excel(io=activityLoc, sheet_name='Pick Task')
df2 = df[df['INDIVIDUAL ITEM WGT'] > 30]
sum = df2['PIECEPICKQTY'].sum()

index = nameMatch('Handling Pick per Case, WeightRangePerCase,0 - 30;')
report['WISE Qty'][index] = sum

In [ ]:
# materials & other charges-pallet charge grade b
df = pd.read_excel(io=activityLoc, sheet_name='Materials')
df2 = df[df['ITEM DESCRIPTION'].str.contains('grade b', case=False, na=False)]
sum = df2['QTY'].sum()

index = nameMatch('materials & other charges-pallet charge grade b')
report['WISE Qty'][index] = sum

In [ ]:
# materials & other charges-stretch wrap
df = pd.read_excel(io=activityLoc, sheet_name='Materials')
df2 = df[df['ITEM DESCRIPTION'].str.contains('stretch wrap', case=False, na=False)]
sum = df2['QTY'].sum()

index = nameMatch('materials & other charges-pallet charge grade b')
report['WISE Qty'][index] = sum

In [ ]:
# storage income initial storage per pallet, locationtype,1 high;palletsize,none;
df = pd.read_excel(io=activityLoc, sheet_name='Receive Task')
sum = df['PALLET QTY'].sum()

index = nameMatch('storage income initial storage per pallet, locationtype,1 high;palletsize,none;')
report['WISE Qty'][index] = sum